<a href="https://colab.research.google.com/github/countifs/worknet/blob/main/Worknet_SMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **♥♥♥ 구인안내문자 출력기 ♥♥♥**

## **※ 사용방법 : [shift+enter] 누르면 설치완료됨 (첫번째 줄은 처음시작할 때만 누르면 됩니다.)**

In [1]:
!sudo pip install --upgrade html5lib==1.0b8
!sudo pip install html_table_parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 889 kB 5.1 MB/s 
  Created wheel for html5lib: filename=html5lib-1.0b8-py3-none-any.whl size=107191 sha256=cbfbe99f16f286216519cde0a8e6812db1caf20cd41a91b54b00e8e74af1649a
  Stored in directory: /root/.cache/pip/wheels/e2/52/c8/5a176112b83d8703a2b66584417fe4c9fb22d981aa2acf5586
Successfully built html5lib
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 3.4 MB/s 
  Created wheel for html-table-parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4645 sha256=c2e8ebfc55b6b4308f534f45ad81ff41e3ef50cd38250528e0b5725bb95875f5
  Stored in directory: /root/.cache/pip/wheels/8b/db/ba/ffbb0a5dcb78

# **1. 자동으로 생성**

## **※ 사용방법 : [Shift+Enter] → 구인인증번호 입력 → [Enter]**
## **※ 주의사항1 : 메시지 출력이 잘 되었는지 꼭 확인하고 사용하세요!**
## **※ 주의사항2 : 오류나면 다시 Shift+Enter 누르세요!**

In [2]:
# 구인인증번호 입력
# URL LINK 만들기
url = 'https://www.work.go.kr/empInfo/empInfoSrch/detail/empDetailAuthView.do?searchInfoType=VALIDATION&callPage=detail&wantedAuthNo='
num = input('\n▶ 구인인증번호를 입력하세요 : ') 
url_link = url+num

print()
print('▶ URL주소 : ', url_link)

import re
import random
import pandas as pd
import numpy as np 
import requests
from html_table_parser import parser_functions
from bs4 import BeautifulSoup
from pprint import pprint

req = requests.get(url_link)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

#채용공고 제목
title = soup.find('p', {'class' : 'tit'})
title_text = title.get_text()

# 회사정보
company_info = soup.select('div.right > div.info > ul > li > div')
com_name = company_info[0].text.strip()
com_type = company_info[1].text.strip()
com_size = '-' if company_info[2].text.strip()=='' else company_info[2].text.strip()
com_year = '-' if company_info[3].text.strip()=='' else company_info[3].text.strip()
com_sales = '-' if company_info[4].text.strip()=='' else company_info[4].text.strip()
com_workers = '-' if company_info[6].text.strip()=='' else company_info[6].text.strip()

# careers-new
job_info = soup.select('div.cont > ul > li > span')
# 임금
job_info_sal = job_info[3].get_text().strip().replace('\n','').replace(' ', '')

# 직무내용 표
data1 = soup.find('div', {'class' : 'careers-table'})
d1_table1 = parser_functions.make2d(data1)

# 직무내용
d1_table1_job_des =  parser_functions.make2d(data1)[1][0].replace(u'*\xa0', '\n*').replace(u'-\xa0', '\n-').replace('\r  ','\n').replace('\r ','\n').replace(u'\xa0', ' ').replace('\r','\n').strip()
job_des = re.sub(' +', ' ', d1_table1_job_des)

# 구직정보 가져오기
data2 = soup.find_all("div", {'class' : 'careers-table v1 center mt20'})

# 모집요강 표1. 경력조건, 학력, 고용형태, 모집인원, 근무예정지 항목으로 구성
table1 = parser_functions.make2d(data2[0])
table1_df = pd.DataFrame(data=table1[1:], columns = table1[0])

# 경력조건
job_con = table1_df['경력조건'][0].replace(u'\xa0', ' ').replace(' ','').replace('\n','')

# 학력
job_edu = table1_df['학력'][0].replace(u'\xa0', ' ').replace(' ','').replace('\n','')
job_type = table1_df['고용형태'][0].replace(u'\xa0', ' ').replace(' ','').replace('\n','')

# 모집인원
job_to_find = table1_df['모집인원'][0].find('\t')
job_to = table1_df['모집인원'][0][:job_to_find]

# 근무지역
job_place = re.sub(' +', ' ', table1_df['근무예정지'][0].replace(u'\xa0', ' ').replace('\n', ' '))
job_place

# 모집요강 표2. 모집직종, 직종키워드, 관련직종 항목으로 구성
table2 = parser_functions.make2d(data2[1])
table2_df = pd.DataFrame(data=table2[1:], columns = table2[0])

# 모집직종
job_name = table2_df.iloc[0,0]

# 근무조건 표. 임금조건, 근무시간, 근무형태, 사회보험, 퇴직급여 항목으로 구성
data3 = soup.find("div", {'class' : 'careers-table center'})
table3 = parser_functions.make2d(data3)
table3_df = pd.DataFrame(data=table3[1:], columns = table3[0])

# 임금조건
job_sal = table3_df.iloc[0,0].replace('\n','')
job_sal = re.sub(' +', ' ', job_sal)

# 근무시간
job_time = table3_df['근무시간'][0].replace(u'\xa0', ' ')
job_time = job_time[:job_time.find('주 소정근로시간')]
job_time = job_time.replace('\n','').replace('\t','')
job_time = re.sub(' +', ' ', job_time).replace('근무시간 협의가능 ','').replace('※ 상세 근무시간 ', '').replace('\r','\n')

job_time2 = job_time[:job_time.find('퇴근')].strip() + job_time[job_time.find('퇴근'):job_time.find('퇴근')*2].strip()
job_time2 = job_time2[:None if job_time2.find('(휴게시간')==-1 else job_time2.find('(휴게시간')]
job_time2 = job_time2[:None if job_time2.find('휴게시간')==-1 else job_time2.find('휴게시간')]
job_time2 = job_time2.replace(' ' ,'')
job_time2

# 근무형태
job_form = table3_df['근무형태'][0]

# 업체명
replace_name = [' ', '주식회사', '(주)', '(유)', '법무법인', '사단법인', '세무법인', '유한책임회사']
num = random.choice(range(2,7,1))*"*"

for i in replace_name:
    com_name = com_name.replace(i, '')
    
if len(com_name) == 2:
    com_name2 = com_name[:1]+num
elif len(com_name) == 3:
    com_name2 = com_name[:1]+num+com_name[-1]
elif len(com_name) == 4:
    com_name2 = com_name[:2]+num+com_name[-1]
else:
    com_name2 = com_name[:2]+num+com_name[len(com_name)-2:len(com_name)]    


print()
print('=======================================================================================')
print()
print('● 공고명 :', title_text)
print()
print('=======================================================================================')
print()

print('● 기업명 :',com_name)
print('● 업종 :', com_type)
print('● 기업규모 :', com_size)
print('● 설립년도 :', com_year)
print('● 연매출액 :', com_sales)
print('● 근로자수 :', com_workers)        
print()
print('=======================================================================================')
print()

print('● 모집직종', job_name, sep = '\n')
print()
print('● 근무예정지', job_place, sep = '\n')
print()
print('● 임금조건(요약)', job_info_sal, sep = '\n')
print()
print('● 임금조건(상세)', job_sal, sep = '\n')
print()
print('● 근무시간', job_time, sep = '\n')
print()
print('● 근무형태', job_form, sep = '\n')
print()
print('● 경력조건', job_con, sep = '\n')
print()
print('● 학력', job_edu, sep = '\n')
print()
print('● 고용형태', job_type, sep = '\n')
print()
print('● 모집인원', job_to, sep = '\n')
print()
print('● 직무내용')
print(job_des,'', sep='\n')
print('=======================================================================================')
print()
print('[0000인력개발센터 구인안내]')
print()
print('■ 업체명 :', com_name2)
print('■ 직종 :', job_name)
print('■ 임금 :', job_info_sal)
print('■ 지역 :', job_place[:[m.start() for m in re.finditer(r" ", job_place)][2]])
print('■ 근무시간 :', job_time2)
print()                                                        
print('"지원희망"이라고 답장 보내주시면 알선해드립니다.')
print('자세한 안내를 희망하시면 전화주세요.')
print('감사합니다^^')
print()
print('☎ 지원문의 : 000 취업상담사 000-0000-0000')



▶ 구인인증번호를 입력하세요 : KF10052210240003

▶ URL주소 :  https://www.work.go.kr/empInfo/empInfoSrch/detail/empDetailAuthView.do?searchInfoType=VALIDATION&callPage=detail&wantedAuthNo=KF10052210240003


● 공고명 : 방문 요양보호사 구인


● 기업명 : 서울종합간호요양센터
● 업종 : 보육시설 운영업
● 기업규모 : 기타
● 설립년도 : -
● 연매출액 : -
● 근로자수 : 39 명


● 모집직종
재가 요양보호사

● 근무예정지
서울특별시 마포구 백범로25길 63, . (염리동, 염리삼성래미안)

● 임금조건(요약)
시급11,520원~11,520원

● 임금조건(상세)
시급 11,520원 이상 ~ 11,520원 이하 임금 협의가능

● 근무시간
(오전) 10시 00분 ~ (오후) 1시 00분 

● 근무형태
주 5일 근무

● 경력조건
관계없음

● 학력
학력무관

● 고용형태
기간의정함이없는근로계약(시간(선택)제)

● 모집인원
1명

● 직무내용
방문요양보호사 구인
염리삼성래미안
주5일 10:00~13:00
5등급 인지활동, 병원 부툭동행


[0000인력개발센터 구인안내]

■ 업체명 : 서울******센터
■ 직종 : 재가 요양보호사
■ 임금 : 시급11,520원~11,520원
■ 지역 : 서울특별시 마포구 백범로25길
■ 근무시간 : (오전)10시00분~(오후)1시00분

"지원희망"이라고 답장 보내주시면 알선해드립니다.
자세한 안내를 희망하시면 전화주세요.
감사합니다^^

☎ 지원문의 : 000 취업상담사 000-0000-0000
